In [14]:
import requests
from io import BytesIO

In [15]:
!pip install dill

Заружаем нашу модель SARIMA вместе с готовой функцией


In [16]:
import dill

with open('/content/drive/MyDrive/ML_3/model (1).pkl', 'rb') as f:
  model_data = dill.load(f)

Разбиваем отдельно на модель отдельно на функцию

In [17]:
model = model_data['model']
predict_N = model_data['predict_N']

In [18]:
model

In [19]:
predict_N

<function __main__.predict_N_weeks(date, current_price, model, steps=6)>

In [20]:
import pandas as pd
import numpy as np

In [21]:
url = "https://github.com/samoletpanfilov/reinforcement_task/raw/refs/heads/master/data/test.xlsx"  # Ссылка должна содержать raw
response = requests.get(url)
test_table = pd.read_excel(BytesIO(response.content), engine='openpyxl')

Предсказывем объем и сохраняем обратно в файл

In [22]:
N_values = []

for date, row in test_table.iterrows():
  N = predict_N(date, row['Цена на арматуру'], model)
  N_values.append(N)

test_table['Объем'] = N_values

In [23]:
test_table.to_excel('sample_submission.xlsx', index=True)

Интерфейс

In [24]:
!pip install gradio

In [26]:
import dill
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time

# Анимированные эмодзи
animated_emojis = {
    'up': '🔼✨',
    'down': '🔽💸',
    'alert': '🚨🚀',
    'chart': '📈🔥',
    'money': '💰🔄'
}

def detect_columns(df):
    """Автоматическое определение колонок с датами и ценами"""
    date_col = None
    price_col = None

    # Ищем колонку с датами
    for col in df.columns:
        # Проверяем тип данных или название колонки
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            date_col = col
        elif 'date' in col.lower() or 'дата' in col.lower():
            date_col = col
            # Попробуем преобразовать в datetime, если это еще не сделано
            try:
                df[col] = pd.to_datetime(df[col])
            except:
                pass

    # Ищем колонку с ценами
    for col in df.columns:
        if 'price' in col.lower() or 'цена' in col.lower() or 'армат' in col.lower():
            price_col = col

    # Если не нашли автоматически, берем первые подходящие колонки
    if date_col is None:
        for col in df.columns:
            if pd.api.types.is_datetime64_any_dtype(df[col]):
                date_col = col
                break

    if price_col is None:
        for col in df.columns:
            if pd.api.types.is_numeric_dtype(df[col]):
                price_col = col
                break

    if date_col is None or price_col is None:
        raise gr.Error("Не удалось автоматически определить колонки с датами и ценами. Убедитесь, что файл содержит колонки с датами и ценами на арматуру.")

    return date_col, price_col

def load_model():
    """Загрузка модели с возвратом статуса"""
    try:
        with open('/content/drive/MyDrive/ML_3/model (1).pkl', 'rb') as f:
            model_data = dill.load(f)
        time.sleep(1)  # Искусственная задержка для эффекта
        return model_data['model'], model_data['predict_N'], f"✅ Модель загружена! {animated_emojis['chart']}"
    except Exception as e:
        raise gr.Error(f"❌ Ошибка загрузки: {str(e)}")

def create_plot(df, date_col, price_col, forecast_col):
    """Создание графика с визуальными эффектами"""
    fig, ax = plt.subplots(figsize=(14, 7))

    # Основные линии
    ax.plot(df[date_col], df[price_col], 'b-', label='Цена', marker='o', markersize=8)
    norm_forecast = (df[forecast_col] / df[forecast_col].max() * df[price_col].max())
    ax.plot(df[date_col], norm_forecast, 'r--', label='Прогноз (норм.)', marker='x', markersize=8)

    # Цветные точки для рекомендаций
    for i, row in df.iterrows():
        color = 'green' if row[forecast_col] == 6 else 'yellow' if row[forecast_col] >=4 else 'red'
        size = 150 if row[forecast_col] ==6 else 100 if row[forecast_col] >=4 else 70
        ax.scatter(row[date_col], row[price_col], c=color, s=size,
                  label=f'Недель роста: {row[forecast_col]}' if i == 0 else "")

    ax.set_title(f'Динамика цен {animated_emojis["chart"]}', fontsize=14, pad=20)
    ax.set_xlabel('Дата', fontsize=12)
    ax.set_ylabel('Цена / Норм. объем', fontsize=12)
    ax.legend(fontsize=12)
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    return fig

def generate_recommendation(N):
    """Генерация рекомендации с эмодзи"""
    if N == 6:
        return f"🚨 СРОЧНАЯ ЗАКУПКА! {animated_emojis['alert']}\nЦена будет расти 6+ недель {animated_emojis['up']}"
    elif N >= 4:
        return f"🟢 Рекомендуется закупиться {animated_emojis['money']}\nЦена будет расти {N} недель {animated_emojis['up']}"
    else:
        return f"🔴 Не закупаться {animated_emojis['down']}\nЦена не будет расти {animated_emojis['down']}"

def predict_with_effects(file, progress=gr.Progress()):
    """Основная функция с обработкой файла"""
    try:
        status_messages = ["⏳ Начинаем анализ..."]

        # Загрузка модели
        model, predict_N, model_status = load_model()
        status_messages.append(model_status)

        # Чтение файла
        test_table = pd.read_excel(file)
        date_col, price_col = detect_columns(test_table)
        status_messages.append("📂 Файл успешно прочитан")

        # Прогнозирование
        test_table['Прогноз_недель'] = 0
        for i in progress.tqdm(range(len(test_table)), desc="Обработка"):
            test_table.loc[i, 'Прогноз_недель'] = predict_N(
                test_table.loc[i, date_col],
                test_table.loc[i, price_col],
                model
            )
            time.sleep(0.05)

        status_messages.append("🔍 Анализ данных завершен")

        # Генерация рекомендаций
        test_table['Рекомендация'] = test_table['Прогноз_недель'].apply(generate_recommendation)
        status_messages.append("📊 Рекомендации сформированы")

        # Создание графика
        fig = create_plot(test_table, date_col, price_col, 'Прогноз_недель')
        status_messages.append("🎨 Визуализация готова")

        # Сохранение результатов
        output_path = '/content/результат_с_эффектами.xlsx'
        test_table.to_excel(output_path, index=False)
        status_messages.append(f"✅ Анализ завершен! {animated_emojis['chart']}")

        return (
            output_path,
            test_table[[date_col, price_col, 'Прогноз_недель', 'Рекомендация']],
            fig,
            "\n".join(status_messages)
        )

    except Exception as e:
        raise gr.Error(f"Ошибка: {str(e)}")

# css
custom_css = """
@keyframes pulse {
  0% { transform: scale(1); }
  50% { transform: scale(1.05); }
  100% { transform: scale(1); }
}
.btn-primary {
    animation: pulse 2s infinite;
    background: linear-gradient(90deg, #4a6baf, #6a8fd8);
    color: white;
    border: none;
}
.status-box {
    background: #f8f9fa;
    padding: 10px;
    border-radius: 5px;
    margin-bottom: 10px;
    text-align: center;
}
"""

with gr.Blocks(theme=gr.themes.Soft(), css=custom_css) as demo:
    gr.Markdown("""
    <div style='text-align: center;'>
    <h1 style='color: #4a6baf;'>🏗️ Smart Steel Advisor</h1>
    <p>Интеллектуальная система прогнозирования цен на арматуру</p>
    </div>
    """)

    with gr.Row(variant="panel"):
        with gr.Column(scale=2):
            gr.Markdown("### 📤 Загрузите данные")
            file_input = gr.File(label="Excel-файл с историей цен",
                               file_types=[".xlsx", ".xls"])
            submit_btn = gr.Button("Запустить анализ", variant="primary", elem_classes=["btn-primary"])

            with gr.Accordion("📚 Вся информация", open=False):
                with gr.Accordion("📝 Инструкция по использованию", open=False):
                    gr.Markdown("""
                    <div style='padding: 10px; background: #ff0000; border-radius: 8px;'>
                    <h4 style='margin-top: 0;'>💡 Основные требования:</h4>
                    <ul>
                        <li>Файл должен содержать колонки с <b>датами</b> и <b>ценами</b> на арматуру</li>
                        <li>Названия колонок определяются автоматически (ищут слова "дата", "цена", "армат")</li>
                        <li>Поддерживаются форматы: <code>.xlsx</code>, <code>.xls</code></li>
                        <li>Минимальный объем данных - 10 записей для точного прогноза</li>
                    </ul>
                    </div>
                    """)

                with gr.Accordion("🎨 Как читать результаты", open=False):
                    gr.Markdown("""
                    <div style='padding: 10px; background: #; border-radius: 8px;'>
                    <h4 style='margin-top: 0;'>📊 Легенда графика:</h4>
                    <ul>
                        <li><span style='color: #28a745; font-weight: bold;'>● Зеленые точки</span> - Срочная закупка (6+ недель роста)</li>
                        <li><span style='color: #ffc107; font-weight: bold;'>● Желтые точки</span> - Рекомендуемая закупка (4-5 недель)</li>
                        <li><span style='color: #dc3545; font-weight: bold;'>● Красные точки</span> - Не закупаться (1-3 недели)</li>
                        <li><span style='color: #007bff; font-weight: bold;'>─ Синяя линия</span> - Фактические цены</li>
                        <li><span style='color: #dc3545; font-weight: bold;'>-- Красная линия</span> - Прогноз (нормализованный)</li>
                    </ul>

                    <h4 style='margin-top: 15px;'>📋 Формат результатов:</h4>
                    <ul>
                        <li>Исходные данные сохраняются полностью</li>
                        <li>Добавляется колонка <code>Прогноз_недель</code> с числом недель роста</li>
                        <li>Колонка <code>Рекомендация</code> содержит текстовый совет</li>
                    </ul>
                    </div>
                    """)

        with gr.Column(scale=3):
            model_status = gr.Textbox(label="Статус системы", value="🟢 Готов к работе",
                                    elem_classes=["status-box"])
            with gr.Tabs():
                with gr.TabItem("📈 График"):
                    plot_output = gr.Plot(label="Динамика цен")
                with gr.TabItem("📋 Таблица"):
                    results_table = gr.Dataframe(
                        label="Рекомендации по закупкам",
                        headers=["Дата", "Цена", "Недель роста", "Рекомендация"],
                        datatype=["str", "number", "number", "str"]
                    )
                with gr.TabItem("💾 Скачать"):
                    file_output = gr.File(label="Полный отчет")

    submit_btn.click(
        fn=predict_with_effects,
        inputs=file_input,
        outputs=[file_output, results_table, plot_output, model_status],
        api_name="predict"
    )

if __name__ == "__main__":
    demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://22ab67fe61538072ca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
